<a href="https://colab.research.google.com/github/haridev98/120-Data-Science-Interview-Questions/blob/master/Bulk_offer_issuance_progress_status_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
headers =\
'''
'''


In [22]:
!pip install tqdm requests pandas==1.5.3 --quiet
import pandas as pd
import requests
import time
from tqdm import tqdm
main_df = pd.DataFrame()

In [23]:
def header_convertor(raw_header)-> dict:
  '''
  Covert raw header to json form
  '''

  header_dict = {}

  for line in raw_header.strip().split('\n'):

    if ': ' in line:

      key, value = line.split(': ', 1)

      header_dict[key] = value

  # returns the dictionary

  return  header_dict

In [24]:
def camapign_detail_requester(id,campaign_header):

  url = "https://mmc-cplotusth.ent-sessionm.com/queries"
  campaign_id = str(id)
  campaign_payload = "{\"query\":\"\\n  query getAdCampaignForm($id: ID!) {\\n    campaign(id: $id) {\\n      id\\n      status\\n      starts_at\\n      ends_at\\n      can_relaunch\\n      name\\n      template\\n      type_data\\n      permalink\\n      behavior_skip_count\\n      user_restrictions\\n      optin_required\\n      opt_in_starts_at\\n      metadata\\n      opt_in_ends_at\\n      auto_enroll\\n      auto_enroll_ttl\\n      auto_enroll_starts_at\\n      auto_enroll_ends_at\\n      group_name\\n      custom_payload\\n      region\\n      organization {\\n        id\\n        name\\n        data\\n        category_set_id: event_category_set_id\\n        applications {\\n          id\\n          name\\n        }\\n        outcome_types\\n      }\\n      owner {\\n        id\\n        full_name\\n        email\\n        avatar_url\\n        deleted\\n      }\\n      contributors {\\n        id\\n        full_name\\n        email\\n        avatar_url\\n      }\\n      approved_by {\\n        id\\n        full_name\\n        email\\n        avatar_url\\n      }\\n      ad_line_items {\\n        id\\n        name\\n        status\\n        ad_groups {\\n          id\\n          name\\n          status\\n          starts_at\\n          ends_at\\n          match_type\\n          ad_units {\\n            id\\n            ad_group_id\\n            name\\n            ssap_name\\n            status\\n            creative_type\\n            data\\n            ad_type\\n            tags\\n            advanced_settings_data\\n            next_ad_unit_id\\n            frequency_cap\\n            frequency_period\\n            frequency_cap_action\\n            application_id\\n            weight\\n            organization_inbox_message {\\n              id\\n              image_id\\n              subject\\n              data\\n            }\\n          }\\n          ad_targets {\\n            id\\n            target_handler\\n            data\\n            realtime\\n            ad_bundles {\\n              id\\n              name\\n              list_type\\n              match_type\\n              ad_targets {\\n                id\\n                target_handler\\n                data\\n                realtime\\n              }\\n              parent_bundle {\\n                id\\n                name\\n                list_type\\n                match_type\\n                ad_targets {\\n                  id\\n                  target_handler\\n                  data\\n                }\\n              }\\n            }\\n          }\\n        }\\n      }\\n      ad_bundle {\\n        id\\n        name\\n        description\\n        list_type\\n        default\\n        match_type\\n        ad_targets {\\n          id\\n          target_handler\\n          data\\n          realtime\\n          ad_bundles {\\n            id\\n            name\\n            list_type\\n            match_type\\n            ad_targets {\\n              id\\n              target_handler\\n              data\\n              realtime\\n            }\\n            parent_bundle {\\n              id\\n              name\\n              list_type\\n              match_type\\n              ad_targets {\\n                id\\n                target_handler\\n                data\\n              }\\n            }\\n          }\\n        }\\n      }\\n    }\\n  }\\n\\n\",\"variables\":{\"id\":\""+ campaign_id +"\"}}\r\n"

  try:
    response = requests.request("POST", url, headers=campaign_header , data= campaign_payload)
    if response.status_code == 200:
      return response.json()
    else:
      print("Error: ", response.status_code)
      print("Content: ", response.content)
  except Exception as e:
    print(f"\nException for campaign(id: {id} ): ", e)



In [25]:
def campaign_details_etl(campaign_id):
  campaign_header = header_convertor(headers)
  main_data =camapign_detail_requester(campaign_id,campaign_header)
  details={}
  flag = False
  details['campaign_id'] = campaign_id
  details['campaign_status'] = main_data['data']['campaign']['status']
  details['campaign_name'] = main_data['data']['campaign']['name']
  ad_line_items =  main_data['data']['campaign']['ad_line_items']
  for data in ad_line_items:
    for data2 in data['ad_groups']:
      for data3 in data2['ad_units']:

       if data3['creative_type'] == 'issue-bulk-offer':
        flag = True
        details['Behavior_name'] = data2['name']
        details['scheduled_delivery_date'] = data3['data']['scheduled_delivery_date']
        details['Adunit_id'] = data3['id']


  if flag:
    return details


  with open("Failed_campaigns.csv",'w') as f:
    f.write(f"Issue Bulk Offer Adunit ID not available for campaign_id: {campaign_id}\n")





In [26]:
def bulk_offer_request(ad_unit_id):
  bulk_offer_header = header_convertor(headers)
  url = "https://mmc-cplotusth.ent-sessionm.com/queries"

  bulk_offer_payload= "{\"query\":\"\\n  query messageAudienceJobs($adUnitID: ID!, $offset: Int, $limit: Int) {\\n    messageAudienceJobs(adUnitID: $adUnitID, offset: $offset, limit: $limit) {\\n      totalCount\\n      edges {\\n        node {\\n          ...audienceJobFragment\\n        }\\n      }\\n    }\\n  }\\n  \\n  fragment audienceJobFragment on AudienceJob {\\n    id\\n    job_handler\\n    account_id\\n    audience_id\\n    audience_size\\n    progress\\n    num_processed\\n    num_ineligible\\n    num_errors\\n    created_at\\n    updated_at\\n    attempted_at\\n    owner {\\n      id\\n      full_name\\n      email\\n      avatar_url\\n    }\\n    ineligible_data_export_instance {\\n      id\\n      status\\n      report_type\\n      expired\\n    }\\n    error_data_export_instance {\\n      id\\n      status\\n      report_type\\n      expired\\n    }\\n    offer\\n  }\\n\\n\\n \",\"variables\":{\"adUnitID\":"+str(ad_unit_id)+",\"offset\":0,\"limit\":5}}"

  try:
    response = requests.request("POST", url, headers=bulk_offer_header , data= bulk_offer_payload)
    if response.status_code == 200:
      return response.json()
    else:
      print("Error: ", response.status_code)
      print("Content: ", response.content)
  except Exception as e:
    print(f"\nException for ad_unit_id {ad_unit_id} : ", e)





In [27]:
def bulk_offer_details_etl(rec):
  if rec is None:
    return
  data = bulk_offer_request(rec['Adunit_id'])
  for data2 in data['data']['messageAudienceJobs']['edges']:
    rec['job_id']= data2['node']['id']
    rec['audience_id']= data2['node']['audience_id']
    rec['audience_size']= data2['node']['audience_size']
    rec[ 'bulk offer issuance progress']= data2['node'][ 'progress']
    rec['num_processed']= data2['node']['num_processed']
    rec['num_ineligible']= data2['node']['num_ineligible']
    rec['num_errors']= data2['node']['num_errors']
    rec['offer_title']= data2['node']['offer']['title']
    rec['offer_external_id']= data2['node']['offer'][ 'external_id']
    record =pd.DataFrame(data=rec,index =['campaign_id'] )
    global main_df
    main_df= pd.concat([main_df ,record ] , ignore_index=True)

In [28]:
def campaigns_summary_report(campaign_id_list ):
  for id in tqdm(campaign_id_list['campaign_id'], desc="Collecting Offer Issuance Data", unit='campaign'):
    time.sleep(0.5)
    bulk_offer_details_etl(campaign_details_etl(id))

  main_df.to_csv("campaigns_summary_report.csv")

In [29]:
if __name__ == '__main__':
  main_df =main_df[0:0]
  campaigns_summary_report(pd.read_csv('campaigns.csv') )


In [30]:
main_df

,campaign_id,campaign_status,campaign_name,Behavior_name,scheduled_delivery_date,Adunit_id,job_id,audience_id,audience_size,bulk offer issuance progress,num_processed,num_ineligible,num_errors,offer_title,offer_external_id
0,13807,completed,CRM_23W44_NAXNAX BANANA CRISPS 100G_3 THB_O11,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,27990,12701,13357,1342,completed,1342,0,0,ลดเพิ่ม 3 บาท สินค้ารู้ใจ,1636a91d-ac9f-475b-aac7-a40e4bb14beb
1,13808,completed,CRM_23W44_NAXNAX BANANA CRISPS 100G_4 THB_O22,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,27992,12692,13358,23558,completed,23558,0,0,ลดเพิ่ม 4 บาท สินค้ารู้ใจ,9853be60-03d7-4e80-a19a-53b44aefeb90
2,13809,completed,CRM_23W44_NAXNAX BANANA CRISPS 100G_4 THB_O56,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,27994,12708,13359,6118,completed,6118,0,0,ลดเพิ่ม 4 บาท สินค้ารู้ใจ,f55ea5c0-1a7b-4f03-952a-579c387eec25
3,13810,completed,CRM_23W44_NAXNAX BANANA CRISPS 100G_4 THB_O67,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,27996,12711,13360,7247,completed,7247,0,0,ลดเพิ่ม 4 บาท สินค้ารู้ใจ,2f75cfd7-e75c-47bf-9f60-3163ee682d48
4,13812,completed,CRM_23W44_NAXNAX BANANA CRISPS 100G_3 THB_O33,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,28000,12691,13362,16801,completed,16801,0,0,ลดเพิ่ม 3 บาท สินค้ารู้ใจ,468fdfe0-81b3-4013-9181-a5903ca286bb
5,13813,completed,CRM_23W44_LOTUSS DISHWASHING LIQUID 500 ML_3 T...,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,28002,12713,13363,282,completed,282,0,0,ลดเพิ่ม 3 บาท สินค้ารู้ใจ,4a2e49e1-44b3-4d0c-9254-f8b2014ffb61
6,13814,completed,CRM_23W44_LOTUSS DISHWASHING LIQUID 500 ML_3 T...,Issue Bulk Offer,2023-10-24T15:00:00.000+00:00,28004,12702,13364,2451,completed,2451,0,0,ลดเพิ่ม 3 บาท สินค้ารู้ใจ,e3a4e766-b2dd-4630-a544-76741dcf59cc
